In [ ]:
from selenium import webdriver
from shutil import which
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException,WebDriverException
import time  
import pandas as pd
import wget
import datetime
from openpyxl import Workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment



def extract_integers(string):
    result = ""
    for char in string:
        if char.isdigit():
            result += char
    return int(result)

#set path for chromedriver extension
chrome_path = which('/Users/karthikpandiri/Downloads/chromedriver_mac_arm64/chromedriver')

service = Service(executable_path=chrome_path)
driver = webdriver.Chrome(service=service)

driver.get('https://www.linkedin.com/')
driver.implicitly_wait(10)


#target username
username = WebDriverWait(driver, 25).until(EC.element_to_be_clickable((By.XPATH, "//*[@name='session_key']")))
password = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//*[@name='session_password']")))

#enter username and password
username.clear()
username.send_keys("pavanbalu.korlepara@gmail.com")
password.clear()
input_password = input("Password: ")
password.send_keys(input_password)

button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[@type='submit']"))).click()

time.sleep(5)
#Creating Workbook

workbook = Workbook()
time.sleep(4)
start = time.time()
current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
filename = f"LinkedIn_{formatted_datetime}.xlsx"
activeSheetNotUsed = True

keywordsList =['Stem Education','Stem Push']
for keyword in keywordsList:
    searchBox = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Search']")))
    searchBox.clear()
    searchBox.send_keys(keyword)
    time.sleep(2)
    searchBox.send_keys(Keys.ENTER)
    time.sleep(4)
    # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//ul[contains(@class,'search-reusables__filter-list')]/li[1]/button[1]")))
    WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Posts')]")))
    Posts_button = driver.find_element(By.XPATH,"//button[contains(., 'Posts')]")
    # Posts_button = driver.find_element(By.XPATH, "//ul[contains(@class,'search-reusables__filter-list')]/li[1]/button[1]")
    Posts_button.click()

    n_scrolls = 1
    for j in range(0, n_scrolls):
        time.sleep(10)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print(j, " Page is scrolled")
    posts = driver.find_elements(By.XPATH, "//div[@class='pt1 mb2 artdeco-card']")

    print('Parsing Begins')
    time.sleep(4)
    
    if activeSheetNotUsed:
        sheet = workbook.active
        sheet.title = keyword + " Data"
        activeSheetNotUsed = False
    else:
        sheet = workbook.create_sheet(keyword + ' Data')
    sheet.column_dimensions['A'].width = 20
    sheet.column_dimensions['B'].width = 130
    sheet.column_dimensions['C'].width = 40
    

    header_labels = ["Usernames", "Descriptions", "HashTags", "Reactions", "Comments", "Reposts"]
    for col_idx, header_label in enumerate(header_labels, start=1):
        cell = sheet.cell(row=1, column=col_idx)
        cell.value = header_label
        cell.font = Font(bold=True,size = 12)
        
        

    for post in posts:
        username = ''
        description = ''
        hashtags = ''
        reactions = 0
        comments = 0
        reposts = 0


        usernames = post.find_elements(By.XPATH,".//span[contains(@class,'update-components-actor__name t-14 t-bold hoverable-link-text')]//span[@dir = 'ltr']")
        for nth_time in range(len(usernames)):
                if nth_time != len(usernames) - 1:
                    username = username + usernames[nth_time].text + ','
                else:
                    username = username + usernames[nth_time].text
    #     username = ",".join(name.text for name in usernames)

        try:
            descriptions = post.find_elements(By.XPATH,".//span[@class='break-words']")
            for nth_time in range(len(descriptions)):
                if nth_time != len(descriptions) - 1:
                    description = description + descriptions[nth_time].text + ','
                else:
                    description = description + descriptions[nth_time].text
            for i in descriptions:
                print(i.text)
        except (NoSuchElementException,WebDriverException):
             pass

    #     description = ",".join(desc.text for desc in descriptions)

        try:
            anchorTags = post.find_elements(By.XPATH,".//span[@class='break-words']//a")
            for nth_time in range(len(anchorTags)):
                if nth_time != len(anchorTags) - 1:
                    hashtags = hashtags + anchorTags[nth_time].get_attribute("textContent") + ','
                else:
                    hashtags = hashtags + anchorTags[nth_time].get_attribute("textContent")
            for tag in anchorTags:
                print(tag.get_attribute("textContent"))
        except (NoSuchElementException,WebDriverException):
            pass

        try:
            reactions = post.find_element(By.XPATH,".//span[@class='social-details-social-counts__reactions-count']")
            print("Reactions-->",reactions.text)
        except (NoSuchElementException,WebDriverException):
            pass

        try:
            comments = post.find_element(By.XPATH,".//button[contains(@class,'t-black--light social-details-social-counts__')]")
            print("Comments-->",comments.text)
        except (NoSuchElementException,WebDriverException):
            pass

        try:
            reposts = post.find_element(By.XPATH,".//button[contains(@class,'ember-view t-black--light')]")
            print("Reposts-->",reposts.text)
        except (NoSuchElementException,WebDriverException):
            pass

        reactions = extract_integers(reactions.text) if reactions != 0 else 0
        comments = extract_integers(comments.text) if comments != 0 else 0
        reposts = extract_integers(reposts.text) if reposts != 0 else 0

    #     if time.time() - start > 78:
    #         break

        sheet.append([username,description,hashtags,reactions,comments,reposts])
        workbook.save(filename)
    font_size = 12  # Adjust the size as desired
    for row in sheet.iter_rows(min_row=2):
        for cell in row:
            cell.font = Font(size=font_size)
    workbook.save(filename)